In [137]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM


In [138]:

# Load the data
df = pd.read_csv('output.csv', header=None)

# # Ignore the even lines
# df = df[df.index % 2 != 0]

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df.values)

# Split the data into training and test sets
train_size = int(len(scaled_data) * 0.8)
train, test = scaled_data[0:train_size,:], scaled_data[train_size:len(scaled_data),:]


In [139]:

def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :]
        dataX.append(a)
        dataY.append(dataset[i + look_back, :])
    return np.array(dataX), np.array(dataY)


In [140]:
# Reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)


In [141]:
# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], trainX.shape[2]))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], testX.shape[2]))

In [142]:
from keras.callbacks import Callback
import numpy as np

class MyCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        target = np.array([12,14,30,35,38,40])  # The target numbers
        prediction = self.model.predict(trainX)  # The model's prediction
        if np.allclose(prediction, target, atol=1e-2):  # If prediction is close to target
            self.model.stop_training = True  # Stop training

In [143]:
# neural network

from keras.models import Sequential
from keras.layers import Dense

# Initialize the constructor
model = Sequential()

# Add an input layer 
model.add(Dense(12, activation='relu', input_shape=(1,6))) 

# Add one hidden layer 
model.add(Dense(8, activation='relu'))

# Add an output layer 
model.add(Dense(6, activation='sigmoid'))  # Update the output shape to (6,)

# FILEPATH: /c:/Users/jomna/Documents/GitHub/LottoNz/lotto_2.ipynb

In [144]:
# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [145]:
# Remove the callback for now
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=1)


Epoch 1/100


ValueError: in user code:

    File "c:\Users\jomna\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\jomna\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\jomna\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\jomna\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\jomna\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "c:\Users\jomna\

In [ ]:
next_numbers = model.predict(scaler.transform(np.array([scaled_data[-1]]).reshape(1, look_back, 6)))
next_numbers = scaler.inverse_transform(next_numbers)

1/1 [==============================] - 0s 34ms/step


In [ ]:
# Round the numbers and convert them to integers
next_numbers = [int(round(num)) for num in next_numbers[0]]

print(f'The predicted next set of numbers is {next_numbers}')


The predicted next set of numbers is [6, 12, 18, 23, 29, 35]
